<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/359_EFIA_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Agent Foundation: State & Configuration Design

This code block defines the **foundation of the Employee Feedback Intelligence Agent**.
It establishes *what the agent knows*, *how it reasons*, and *how it remains transparent and controllable* as it turns employee feedback into executive insight.

Rather than jumping straight into AI or automation, the system starts with something far more important: **a clear, auditable state model and an explicit configuration layer**.

This design choice is intentional.

---

## Why This Matters

Most AI systems hide their logic inside black-box prompts or opaque pipelines.
This agent does the opposite.

From the start, it:

* Makes every intermediate result visible
* Separates data, reasoning, and presentation
* Keeps business logic explicit and configurable
* Uses AI only where it adds real value

That’s what makes this system trustworthy at scale.

---

## The Agent State: A Single Source of Truth

### `EmployeeFeedbackIntelligenceState`

The `EmployeeFeedbackIntelligenceState` defines the **entire lifecycle of the agent’s reasoning** — from raw employee feedback to an executive-ready report.

Think of it as the agent’s “working memory.”

At any point, you can inspect the state and answer questions like:

* What data was loaded?
* How was it grouped?
* What themes were detected?
* Why were certain issues prioritized?
* What summaries were generated?

Nothing is hidden.

---

### What the State Tracks (Conceptually)

Instead of raw logs or scattered variables, the state captures **business-relevant milestones**:

#### 1. Input & Planning

* Where feedback data comes from
* What the agent’s goal is
* What execution plan it follows

This ensures the agent always operates with **clear intent**, not improvisation.

---

#### 2. Data Ingestion

* All employee feedback entries loaded from JSON files
* Consistent structure across departments

This creates a clean, auditable dataset before any analysis begins.

---

#### 3. Aggregation & Trends

* Feedback grouped by department
* Feedback grouped by category (Issues vs Ideas)
* High-level counts and distributions

This is where raw comments become **organizational signal**.

---

#### 4. Sentiment Analysis

* Sentiment classification per submission
* Intensity scoring
* Aggregated sentiment by department and category

Importantly, sentiment is treated as **supporting evidence**, not the sole decision driver.

---

#### 5. Theme Detection

* Recurring issues and ideas grouped into themes
* Frequency, sentiment, and departmental impact tracked

This allows leadership to see **patterns**, not anecdotes.

---

#### 6. Prioritization (Pareto Logic)

* Issues and ideas ranked by business relevance
* Rationale stored alongside scores
* Explicit reasoning for why something matters

This is where the agent moves from analysis to **decision support**.

---

#### 7. LLM-Based Summaries

* Department-level summaries
* Category-level summaries
* Theme summaries
* A single executive summary

Crucially, the LLM does not decide priorities —
it **explains conclusions the system has already reached**.

---

#### 8. Visual Outputs

* Paths to saved charts and visuals
* Clear mapping between insights and artifacts

This makes reporting repeatable and presentation-ready.

---

#### 9. Outputs & Metadata

* Final markdown report
* Processing time
* Error tracking

This supports reliability, debugging, and future performance tracking.

---

## Configuration: Control Without Code Changes

### `EmployeeFeedbackIntelligenceConfig`

The configuration class defines **how the agent behaves** — without touching logic.

This is a critical design decision for business adoption.

---

### What Leaders and Operators Gain

Through configuration, you can:

* Change which data files are used
* Adjust how many issues or ideas are highlighted
* Tune sentiment sensitivity
* Control theme thresholds
* Enable or disable LLM summarization
* Standardize visual output formats

All without rewriting the agent.

This is how the system remains:

* Flexible
* Auditable
* Governable
* Safe to iterate

---

### Rules Before AI

A key principle is visible throughout the config:

* Sentiment uses keyword rules
* Themes require minimum frequency
* Prioritization is explicit
* Thresholds are documented

AI enhances the system — it does not obscure it.

This keeps decision-making **explainable**, which is essential for executive trust.

---

## Architectural Takeaway

This first block doesn’t “do” analysis yet — and that’s the point.

It defines:

* The contract for reasoning
* The boundaries of automation
* The structure for accountability
* The foundation for executive confidence

Before the agent analyzes a single piece of feedback, it already answers an important question:

> “If a leader asks *why* this system reached a conclusion — can we show them?”

With this design, the answer is **always yes**.



# Employee Feedback Intelligence Agent

In [ ]:
# ============================================================================
# Employee Feedback Intelligence Agent
# ============================================================================

class EmployeeFeedbackIntelligenceState(TypedDict, total=False):
    """State for Employee Feedback Intelligence Agent"""

    # Input fields
    data_dir: Optional[str]                 # Directory containing feedback JSON files
    # Default: "agents/data" (fulfillment.json, general_merchandise.json, guest_advocate.json)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                    # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Data Ingestion
    raw_feedback: List[Dict[str, Any]]      # All feedback entries loaded from JSON files
    # Structure per entry:
    # {
    #   "submission_id": "FE-001",
    #   "timestamp": "2025-12-11 09:10",
    #   "job_area": "Fulfillment Expert",
    #   "category": "Issue" | "Idea",
    #   "employee_name": "" | "Jordan",
    #   "free_text_feedback": "..."
    # }

    # Aggregation & Trend Detection
    feedback_by_department: Dict[str, List[Dict[str, Any]]]  # Department -> list of feedback
    feedback_by_category: Dict[str, List[Dict[str, Any]]]   # "Issue" | "Idea" -> list
    feedback_summary: Dict[str, Any]          # Overall summary counts
    # Structure:
    # {
    #   "total_feedback": 70,
    #   "total_issues": 40,
    #   "total_ideas": 30,
    #   "departments": ["Fulfillment Expert", "General Merchandise Expert", "Guest Advocate"],
    #   "feedback_by_department": {
    #     "Fulfillment Expert": 25,
    #     "General Merchandise Expert": 25,
    #     "Guest Advocate": 20
    #   }
    # }

    # Sentiment Analysis
    sentiment_analysis: List[Dict[str, Any]]  # Sentiment per feedback entry
    # Structure per entry:
    # {
    #   "submission_id": "FE-001",
    #   "sentiment": "negative" | "neutral" | "positive",
    #   "intensity": float,  # 0.0-1.0
    #   "sentiment_score": float  # -1.0 to 1.0
    # }

    sentiment_summary: Dict[str, Any]        # Aggregated sentiment metrics
    # Structure:
    # {
    #   "overall_sentiment": "negative" | "neutral" | "positive",
    #   "positive_count": 10,
    #   "neutral_count": 20,
    #   "negative_count": 40,
    #   "average_intensity": 0.65,
    #   "sentiment_by_department": {...},
    #   "sentiment_by_category": {...}
    # }

    # Theme Detection & Grouping
    themes: List[Dict[str, Any]]            # Detected themes across feedback
    # Structure per theme:
    # {
    #   "theme_id": "theme_001",
    #   "theme_name": "Device/Technology Issues",
    #   "category": "Issue",
    #   "frequency": 8,
    #   "departments": ["Fulfillment Expert"],
    #   "example_feedback": ["Zebra devices freeze...", "..."],
    #   "sentiment": "negative",
    #   "average_intensity": 0.75
    # }

    # Prioritization (80/20 Pareto Analysis)
    prioritized_issues: List[Dict[str, Any]]  # Top issues by priority score
    prioritized_ideas: List[Dict[str, Any]]   # Top ideas by priority score
    # Structure per prioritized item:
    # {
    #   "submission_id": "FE-001",
    #   "theme_id": Optional["theme_001"],
    #   "priority_score": 85.5,
    #   "frequency": 8,  # If part of theme
    #   "sentiment_intensity": 0.75,
    #   "department": "Fulfillment Expert",
    #   "category": "Issue",
    #   "rationale": "High frequency (8 occurrences) with strong negative sentiment"
    # }

    prioritization_summary: Dict[str, Any]   # Summary of prioritization
    # Structure:
    # {
    #   "top_3_issues": [...],
    #   "top_3_ideas": [...],
    #   "issues_driving_80_percent": [...],  # Issues that drive 80% of friction
    #   "ideas_with_highest_impact": [...]
    # }

    # LLM-Based Summarization
    department_summaries: Dict[str, str]     # Department -> summary text
    category_summaries: Dict[str, str]       # "Issue" | "Idea" -> summary text
    theme_summaries: Dict[str, str]          # theme_id -> summary text
    executive_summary: str                    # Overall executive summary

    # Visualizations
    visualization_paths: Dict[str, str]      # Paths to saved chart images
    # Structure:
    # {
    #   "issues_by_department": "output/employee_feedback/charts/issues_by_department.png",
    #   "ideas_by_department": "output/employee_feedback/charts/ideas_by_department.png",
    #   "sentiment_by_department": "output/employee_feedback/charts/sentiment_by_department.png",
    #   "top_issues": "output/employee_feedback/charts/top_issues.png",
    #   "top_ideas": "output/employee_feedback/charts/top_ideas.png"
    # }

    # Output
    feedback_intelligence_report: str        # Final markdown report
    report_file_path: Optional[str]          # Path to saved report file

    # Metadata
    errors: Annotated[List[str], operator.add]  # Any errors encountered
    processing_time: Optional[float]         # Time taken to process


@dataclass
class EmployeeFeedbackIntelligenceConfig:
    """Configuration for Employee Feedback Intelligence Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/employee_feedback_reports"  # Where to save reports
    charts_dir: str = "output/employee_feedback/charts"    # Where to save charts

    # Data file paths
    data_dir: str = "agents/data"
    feedback_files: List[str] = field(default_factory=lambda: [
        "fulfillment.json",
        "general_merchandise.json",
        "guest_advocate.json"
    ])

    # Prioritization Settings
    # Note: Uses Pareto (80/20) principle conceptually - identifies the small number of
    # issues/ideas that drive the majority of impact, but no strict threshold needed
    top_n_issues: int = 10                   # Number of top issues to highlight
    top_n_ideas: int = 10                    # Number of top ideas to highlight

    # Sentiment Analysis Settings
    sentiment_keywords: Dict[str, List[str]] = field(default_factory=lambda: {
        "negative": ["frustrated", "impossible", "difficult", "slow", "broken", "problem", "issue", "hard", "pressure", "overwhelmed"],
        "positive": ["great", "good", "helpful", "improve", "better", "excellent", "love", "appreciate"],
        "neutral": []  # Neutral if neither negative nor positive keywords found
    })

    # Theme Detection Settings
    min_theme_frequency: int = 3             # Minimum occurrences to form a theme
    theme_similarity_threshold: float = 0.6   # Similarity threshold for grouping (future: LLM-based)

    # LLM Summarization Settings
    enable_llm_summarization: bool = True    # Enable LLM-based summarization
    max_summary_tokens: int = 500            # Max tokens per summary
    summarize_top_n_themes: int = 5          # Number of top themes to summarize

    # Visualization Settings
    chart_width: int = 10                    # Chart width in inches
    chart_height: int = 6                    # Chart height in inches
    chart_dpi: int = 100                     # Chart resolution


